In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options

In [2]:
CHROME_PATH = "/Users/ik/bin/chromedriver"

In [3]:
baseURL = "https://www.blinkfitness.com/locations/"
branch = "jamaica" #hard-coded for now
URL = baseURL + branch

In [4]:
# initialize web driver with eager page loading strategy
chrome_options = Options()
chrome_options.page_load_strategy = 'eager'
driver = webdriver.Chrome(CHROME_PATH, options=chrome_options)
driver.get(URL)

In [5]:
# get branch information (location, phone number, and email) through class selection
location_info = WebDriverWait(driver, timeout=15).until(lambda d: d.find_element_by_class_name('Core-address'))

In [6]:
info_dict = {}

# parse coordinates
coord_info = location_info.find_element_by_class_name('coordinates')
_coords = coord_info.find_elements_by_tag_name('meta')

for ele in _coords:
    attr = ele.get_attribute('itemprop').strip()
    content = ele.get_attribute('content').strip()
    
    info_dict[attr] = content
    
# parse location address
address = location_info.find_element_by_class_name('c-address').text.strip()
info_dict['address'] = address.replace('\n',', ')

print(info_dict)

{'latitude': '40.704465555892526', 'longitude': '-73.7966098925308', 'address': '163-02 Jamaica Avenue, Queens, NY 11432'}


In [7]:
# get branch hours element by class name (not most reliable)
hours_info = WebDriverWait(driver, timeout=15).until(lambda d: d.find_element_by_class_name('c-hours-details-wrapper'))

In [8]:
# literal_evel safely evaluate an expression node or a string containing a Python expression
from ast import literal_eval

# days and hours are stored in dictionary stored in data-days attribute of HTML element
hours_dict = literal_eval(hours_info.get_attribute('data-days'))
print(hours_dict)

[{'day': 'TUESDAY', 'intervals': [{'end': 2100, 'start': 600}]}, {'day': 'WEDNESDAY', 'intervals': [{'end': 1900, 'start': 600}]}, {'day': 'THURSDAY', 'intervals': [{'end': 1300, 'start': 700}]}, {'day': 'FRIDAY', 'intervals': [{'end': 1900, 'start': 700}]}, {'day': 'SATURDAY', 'intervals': [{'end': 1600, 'start': 700}]}, {'day': 'SUNDAY', 'intervals': [{'end': 1600, 'start': 700}]}, {'day': 'MONDAY', 'intervals': [{'end': 2100, 'start': 600}]}]


In [ ]:
# used to convert day (string) to integer representation for efficient storage
day_dict = {
    'sunday': 0,
    'monday': 1,
    'tuesday': 2,
    'wednesday': 3,
    'thursday': 4,
    'friday': 5,
    'saturday': 6
}

# parse business hours
for entry in hours_dict:
    
    day = day_dict[entry['day'].lower()]
    
    # TODO: store start_time string as time in sqlite
    start_time = entry['intervals'][0]['start'] 
    end_time = entry['intervals'][0]['end'] 
    
    print(day,start_time,end_time)

In [ ]:
for spec in content:
    print("-----")
    print(spec.text)
    daysOfWeek = spec.find_element_by_xpath('//span[@itemprop="dayOfWeek"]')
    open_hrs = spec.find_element_by_xpath('//time[@itemprop="opens"]')
    close_hrs = spec.find_element_by_xpath('//time[@itemprop="closes"]')
    
    print(daysOfWeek.text)
    print(open_hrs.get_attribute('content'), close_hrs.get_attribute('content'))